In [21]:
%run ipy_setup.py
%matplotlib inline

import sys
sys.path.insert(0, '/hpc/users/neffr01/zhang_neffr01/avggin/')
from __init__ import *
import database
import networkx as nx

In [47]:
db.session.rollback()
netlist = []
for a in ggi_node.query.filter_by(label="APP").all():
    if "onlyclust" in a.ggi_network.properties:
        netlist.append(a.ggi_network)
        print(a.ggi_network.properties)
        #print([(e.source_node.label,e.target_node.label) for e in a.edges().all()])

{'name': '6_ROSMAP.onlyclust.graphml_signature'}
{'name': '2_ROSMAP.onlyclust.graphml_signature'}
{'name': '5_ROSMAP.onlyclust.graphml_signature'}
{'name': '10_ROSMAP.onlyclust.graphml_signature'}
{'name': '4_ROSMAP.onlyclust.graphml_signature'}
{'name': '13_ROSMAP.onlyclust.graphml_signature'}
{'name': '12_ROSMAP.onlyclust.graphml_signature'}
{'name': '18_ROSMAP.onlyclust.graphml_signature'}
{'name': '16_ROSMAP.onlyclust.graphml_signature'}
{'name': '8_ROSMAP.onlyclust.graphml_signature'}
{'name': '15_ROSMAP.onlyclust.graphml_signature'}
{'name': '9_ROSMAP.onlyclust.graphml_signature'}
{'name': '3_ROSMAP.onlyclust.graphml_signature'}
{'name': '7_ROSMAP.onlyclust.graphml_signature'}


In [20]:
netlist[0]

<ggi_network 27>

In [24]:
combined_graph = nx.Graph()
edges_traversed = defaultdict(dict)
count = 0
for net in netlist:
    edgelist = net.edges.all()
    for edge in edgelist:
        node1=edge.source_node
        node2=edge.target_node
        node1_all = None
        node2_all = None
        if node1.ensembl_id != None:
            node1_all = ggi_node.query.filter_by(ensembl_id=node1.ensembl_id).all()
        else:
            node1_all = ggi_node.query.filter_by(label=node1.label).all()
        if node2.ensembl_id != None:
            node2_all = ggi_node.query.filter_by(ensembl_id=node2.ensembl_id).all()
        else:
            node2_all = ggi_node.query.filter_by(label=node2.label).all()
        break
    break

In [46]:
print(net.id)
print([a.ggi_network_id for a in ggi_edge.query.filter(ggi_edge.source.in_([n.id for n in node1_all]), 
                      ggi_edge.target.in_([n.id for n in node2_all]),
                      ggi_edge.ggi_network_id.in_([n.id for n in netlist])).all()])

print([a.ggi_network_id for a in ggi_edge.query.filter(ggi_edge.target.in_([n.id for n in node1_all]),
                     ggi_edge.source.in_([n.id for n in node2_all]),
                     ggi_edge.ggi_network_id.in_([n.id for n in netlist])).all()])

27
[27, 21]
[]


In [ ]:
'''
combine all networks - ryan neff

idea: add values for all edges from each network, 1 network at a time, 
getting all edges for that pair of nodes from the DB as each edge is seen

if an edge was seen, it gets added to edges_traversed and not checked again

'''

db.session.rollback()
netlist = []
for a in ggi_network.query.all():
    if "onlyclust" in a.properties:
        netlist.append(a)
        print(a.properties)

combined_graph = nx.Graph()
edges_traversed = set()
#count = 0
print(len(netlist))
for net in netlist:
    edgelist = net.edges.all()
    pbar = ProgressBar(len(edgelist))
    print("Working on: %s"%net.properties)
    for edge in edgelist:
        pbar.animate()
        node1=edge.source_node
        node2=edge.target_node
        if tuple(sorted([node1.label,node2.label])) in edges_traversed:
            continue #speedup for edges already analyzed
        else:
            edges_traversed.add(tuple(sorted([node1.label,node2.label])))
        node1_all = None
        node2_all = None
        if node1.ensembl_id != None:
            node1_all = ggi_node.query.filter_by(ensembl_id=node1.ensembl_id).all()
        else:
            node1_all = ggi_node.query.filter_by(label=node1.label).all()
        if node2.ensembl_id != None:
            node2_all = ggi_node.query.filter_by(ensembl_id=node2.ensembl_id).all()
        else:
            node2_all = ggi_node.query.filter_by(label=node2.label).all()
        
        all_edge = ggi_edge.query.filter(ggi_edge.source.in_([n.id for n in node1_all]), 
                      ggi_edge.target.in_([n.id for n in node2_all]),
                      ggi_edge.ggi_network_id.in_([n.id for n in netlist])).all()
        
        all_edge.extend(ggi_edge.query.filter(ggi_edge.target.in_([n.id for n in node1_all]), 
                     ggi_edge.source.in_([n.id for n in node2_all]),
                     ggi_edge.ggi_network_id.in_([n.id for n in netlist])).all())
        
        num_edges = len(all_edge)
        subnets = [(a.ggi_network_id,a.ggi_network.properties) for a in all_edge]
        
        #add edge to network
        combined_graph.add_edge(node1.label,node2.label,{"weight":num_edges,"subnets":subnets})
        
        # idea - join the ggi_edge and ggi_node tables (how to do this?)

{'name': '6_ROSMAP.onlyclust.graphml_signature'}
{'name': '2_ROSMAP.onlyclust.graphml_signature'}
{'name': '5_ROSMAP.onlyclust.graphml_signature'}
{'name': '10_ROSMAP.onlyclust.graphml_signature'}
{'name': '4_ROSMAP.onlyclust.graphml_signature'}
{'name': '13_ROSMAP.onlyclust.graphml_signature'}
{'name': '12_ROSMAP.onlyclust.graphml_signature'}
{'name': '18_ROSMAP.onlyclust.graphml_signature'}
{'name': '16_ROSMAP.onlyclust.graphml_signature'}
{'name': '8_ROSMAP.onlyclust.graphml_signature'}
{'name': '15_ROSMAP.onlyclust.graphml_signature'}
{'name': '9_ROSMAP.onlyclust.graphml_signature'}
{'name': '3_ROSMAP.onlyclust.graphml_signature'}
{'name': '7_ROSMAP.onlyclust.graphml_signature'}
14
Working on: {'name': '6_ROSMAP.onlyclust.graphml_signature'}
 [****************100%******************]  46515 of 46655 complete Elapsed: 10.9 minutes	 Remaining: 0.0 minutesWorking on: {'name': '2_ROSMAP.onlyclust.graphml_signature'}
 [****************100%******************]  46148 of 46333 complete Elap

In [ ]:
nx.write_graphml(combined_graph,"/hpc/users/neffr01/zhang_neffr01/avggin/combined_network.onlyclust.graphml")